## Line Detection Pipeline: 
# This part of the pipeline is for extracting lines from videos.    

As usual, edge detection is always a challenge.  A variety of edge types and color spaces
were used to create edges.  The code also includes the perspective transformation that
change the view to bird's eye view.  

All edge functions are in the file edge_utils,py.  Hence, import edge_utils as eu

As in the previous lesson, moviepy is used to read and write mpg files.
MoviePy utilizes an iterator method called process_image.  The color edge detection
occurs in this method.

Both versions operate on images in a source directory (src_dir) store the results in a an output directory (out_dir), as below.

First we can run some tests to see which colorspaces and colorchannels give the best contrast for edge detection.

Run the cell below to output the images


In [ ]:
from moviepy.editor import VideoFileClip

import numpy as np
import glob
import edge_utils as eu

import pickle

from moviepy.editor import *
import cv2

def computeEdges(binary):   
    '''
    Three edge detection methods based on Sobel were used.
    The results of each were accumlated in the numpy array summer
    Thus, in this case summer could have a max of 3 for an
    edge pixel that was detected in all three methods.  Thresholds
    were determined by trial and error
    ''' 
    summer = eu.dir_threshold(binary, 3, thresh=(.7,1.4)) 
    summer = np.add(summer, eu.abs_sobel_thresh(binary, 'x', .2,1.2))  
    summer = np.add(summer, eu.abs_sobel_thresh(binary, 'y', .4 ,1.1)) 

   # dilate = cv2.dilate(summer,dilation_kernel,iterations = 3) 
    print("max at end of edge summation {}".format(np.max(summer)))
    return summer


def process_image(image):
  '''
  This method is called once per image by MoviePy. 
  Three colorspace and channels were tested-a white, yellow
  colormask (color_mask), HLS channel S and HLS L, XYZ X
  As with the edge detection, pixels are accumulated in s.
  s is thresholded later.  
  ''' 
  s = computeEdges(eu.color_mask(image)) 

  HLS = cv2.cvtColor(image, cv2.COLOR_BGR2HLS)
  #s = np.add(s,computeEdges(HLS[:,:,0]))

  #s = np.add(s, computeEdges(HLS[:,:,1]))  

  s = np.add(s, computeEdges(HLS[:,:,2]))  
  XYZ = cv2.cvtColor(image, cv2.COLOR_BGR2XYZ)
  s = np.add(s, computeEdges(XYZ[:,:,0]))
  
  #the uniques (ranging 0:5 typically) are thresholded
  summer_max = np.max(np.unique(s))
  print("summer max {}".format(summer_max))
 
  #less than 10% pixels provide good signal
  for level in np.unique(s):
    percent_active = np.count_nonzero(s>level, axis=None)/np.size(s)
    print("level {}; %pix {}".format(level,percent_active))
    if percent_active < .10:
        break
          
  print("threshold {}".format(level))
  s[s<=level] = 0
  s[s>level] = 1 

  print("percent activated pixels {}".
       format(np.count_nonzero(s, axis=None)/np.size(s)))

  morphed = cv2.dilate(s,dilation_kernel,iterations = 1) 
  print("max of morphed is {}".format(np.max(morphed)))

  warped = cv2.warpPerspective(morphed, M, (1280,720)) 
  print(np.max(warped))

  return np.dstack((255*warped,255*warped,255*warped))



pts = np.array([[581,477],[699,477],[896,675],[384,675]],dtype = "float32")
dst = np.array([[384,0],[896,0],[896,720],[384,720]],dtype = "float32")
M = cv2.getPerspectiveTransform(pts, dst)

with open('camera.pickle', 'rb') as handle:
    (dist, mtx) = pickle.load(handle)

erosion_kernel = np.ones((2,2),np.uint8)
dilation_kernel = np.ones((2,2), np.uint8)

src_dir = './calibrated_test_images/'
out_dir = './birdseye_edge_test_images/'

#for solidWhiteRight, uncomment the following lines
#if they are not uncommented already
movies = glob.glob(src_dir+'*.mp4')

for movie in movies:
   print(movie)
   outfile = movie.replace(src_dir,out_dir)
   
   #input clip call the process_image iterator one
   #time per frame
   input_clip = VideoFileClip(movie)
   
   #to run only a segment of clips, use below
   #arguments are in seconds
   #start = input_clip.duration/2
   #input_clip = input_clip.subclip(start, start+10)
   

   #print(type(process_image))

   clip = input_clip.fl_image(process_image) 
   clip.write_videofile(outfile, audio=False)




./calibrated_test_images/harder_challenge_video.mp4
max at end of edge summation 2
max at end of edge summation 1
max at end of edge summation 1
summer max 4
level 0; %pix 0.5330946180555556
level 1; %pix 0.2516872829861111
level 2; %pix 0.01573025173611111
threshold 2
percent activated pixels 0.01573025173611111
max of morphed is 1
1
[MoviePy] >>>> Building video ./birdseye_edge_test_images/harder_challenge_video.mp4
[MoviePy] Writing video ./birdseye_edge_test_images/harder_challenge_video.mp4






  0%|          | 0/1200 [00:00<?, ?it/s]

max at end of edge summation 2


max at end of edge summation 1






  0%|          | 1/1200 [00:00<08:45,  2.28it/s]

...

max at end of edge summation 2
summer max 4
level 0; %pix 0.6008376736111111
level 1; %pix 0.3000998263888889
level 2; %pix 0.029810112847222223
threshold 2
percent activated pixels 0.029810112847222223
max of morphed is 1
1


max at end of edge summation 2
max at end of edge summation 2






 61%|██████    | 733/1200 [06:00<04:12,  1.85it/s]

max at end of edge summation 1
summer max 4
level 0; %pix 0.5945627170138889
level 1; %pix 0.2991178385416667
level 2; %pix 0.03048828125
threshold 2
percent activated pixels 0.03048828125
max of morphed is 1
1


max at end of edge summation 2
max at end of edge summation 1


## Output Images for Examination
The cell below was used to output many colorspace and channels from the test images. The images were useful for deciding which combination of channels might provide good edges.

In [ ]:
import cv2
import numpy as np
import glob
import edge_utils as eu


def write_images(img, fname, channels, cspace):

    """ 
    write_images:  this function results in a series of images
    being written to an output directory.  It is useful for testing
    color channels and spaces that might be of interest.  The channels
    of interest and the colorspace are channels and cspace.  fname
    represents the base name of the file e.g., test1.jpg
    """

    for channel in channels:
       if (len(np.shape(img))==2):
         outimg = img
       else:
         outimg = img[:,:,channel]
       #note:  histogram equalizaiton using local tiles was tried
       #the may be benefit to equalizing some color channels
       #image = img.astype(np.uint8)
       #clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
       #cl = clahe.apply(outimg)
       #dst = np.zeros_like(outimg)
       #cv2.equalizeHist( outimg, dst );  
       
        #make an output name for the channel image
       outname = fname.replace(src_dir, out_dir)
            
       cv2.imwrite(outname.replace('.jpg', 
                   cspace + '_' + cspace[channel] + '.jpg') ,
                   outimg)
 

src_dir = "./test_images/"
out_dir = './color_channel_test_images/'
infiles = glob.glob(src_dir+'*.jpg')


##loop through the images, writing out the various important color channels

for file in infiles:
  print(file)
  img = cv2.imread(file)
  c_mask = eu.color_mask(img)
  write_images(c_mask, file, [0], 'CMASK')
  write_images(cv2.cvtColor(img, cv2.COLOR_BGR2HLS), file, [0,1,2], 'HLS')
  write_images(cv2.cvtColor(img, cv2.COLOR_BGR2GRAY), file, [0], 'GRY')

  write_images(cv2.cvtColor(img, cv2.COLOR_BGR2HSV), file, [1,2], 'HSV')
  #write_images(cv2.cvtColor(img, cv2.COLOR_BGR2LAB), file, [2], 'LAB')

  write_images(cv2.cvtColor(img, cv2.COLOR_BGR2YCrCb), file, [0], 'YCC') 

  write_images(cv2.cvtColor(img, cv2.COLOR_BGR2LUV), file, [2], 'LUV')
  write_images(cv2.cvtColor(img, cv2.COLOR_BGR2XYZ), file, [0], 'XYZ')
        
